In [1]:
import time
import json
import ast
import os
import datetime
import io
from collections import defaultdict

import imageio
import boto3
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
import seaborn as sns

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
from timezonefinder import TimezoneFinderL
import pytz

from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
%matplotlib inline
pd.set_option('display.max_rows', 500)

/home/alexli/miniconda3/envs/cvml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
athena = AthenaClient()
s3 = boto3.resource('s3')
tf = TimezoneFinderL()
home = os.path.expanduser('~')
outpath = os.environ['OUTPUT_PATH']
data_path = home + '/data'

In [3]:
df_cache = outpath + "/shortlong_exposure.csv"
try:
    skip_df = pd.read_csv(df_cache)
except FileNotFoundError:
    print("file not found")
    query = f"""
    SELECT
        image_jupiter.id, image_jupiter.camera_location AS camera_location, json_extract(image_jupiter.calibration_data__json, '$.exposure_ms') exposure_ms
    FROM image_jupiter
    WHERE
        image_jupiter.camera_location IN ('T01', 'T02', 'T03', 'T04', 'T05', 'T06', 'T07', 'T08', 'T09', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'I01', 'I03', 'I05', 'I07', 'I02', 'I04', 'I06', 'I08')
        AND (cast(json_extract_scalar(image_jupiter.calibration_data__json, '$.exposure_ms.1') AS double)  > 0.2
        OR cast(json_extract_scalar(image_jupiter.calibration_data__json, '$.exposure_ms.1') AS double)  < 0.02)
    LIMIT 1000
    """
    start = time.time()
    skip_df = athena.get_df(query)
    end = time.time()
    print(end - start)
    skip_df.to_csv(df_cache, index=False)

In [7]:
from brtdevkit.data import Dataset
dataset_name = "high_low_exposure"
dataset_description = "Images with a very high or very low exposure time"
dataset_kind = Dataset.KIND_IMAGE
image_ids = imageids_to_dataset(list(skip_df['id']), dataset_name, dataset_description, dataset_kind=Dataset.KIND_IMAGE, mode='stereo')

In [5]:
image_ids

['652e2bd23cc79d0f919a2cff',
 '64eea2415e01413330f20720',
 '6527d7138714c2934b6a701f',
 '6540558c34ac8b994dd1036c',
 '652ffe89c306f143a1769fdc',
 '651f2d2fd8e7e8d575d47a53',
 '6515690a4cf432ab4ae93db2',
 '64f114ac328672a750a5c0d3',
 '64b3d1b82267229a9b32e23a',
 '64b360c14304a51cad48fd5d',
 '64f17f30adb96901703b9f98',
 '65277beb2f0e60426f4d6dd9',
 '653759ff5d7de481a2604d07',
 '653f68fa685b1a4f1441d2a0',
 '653f9044b7561f2f2fcb67c4',
 '6540b4c307137ca9326001af',
 '652e42542eafda4446180a8f',
 '652484e0fb84e372139cf30d',
 '652e5ecd4010120c1ccab802',
 '64b48485fb6f40da384be08b',
 '652999853a2f7579adc331a7',
 '64f141fe8ab510769125a435',
 '6503d8c90b357adbe433df8a',
 '64b3d1ba3eae54bd409791af',
 '65312e2f0c55b915235885fe',
 '65207f9f1f9de7938a196976',
 '652f085583bebd0c4c47b959',
 '64ae419dfad541cef5d6f5a6',
 '653f46d0e0e83a2761a576da',
 '653aed5a5ba9662fe39ded7e',
 '65307e8dd1804f782d5d8efd',
 '652f13ce891ab346a5bea7cd',
 '6441f6522f011935dbec5b3c',
 '6504e0062760cd80b1f4d083',
 '6455c452bd78

In [12]:
Dataset.create(
    name=dataset_name,
    description=dataset_description,
    kind=Dataset.KIND_IMAGE,
    image_ids=image_ids,
)

{}

In [2]:
import brtdevkit
print(brtdevkit.__version__) # '7.10.5'
from brtdevkit.data import Dataset
Dataset.create(
    name="alex_test_dataset_ok",
    description="",
    kind=Dataset.KIND_IMAGE,
    image_ids=['652e2bd23cc79d0f919a2cff'],
) # Returns {}
Dataset.retrieve(name='alex_test_dataset_ok')

7.10.5


InvalidRequestError: no matching resource found with given parameter(s)

{}

In [16]:
Dataset.retrieve(name='alex_test_dataset_ok')

InvalidRequestError: no matching resource found with given parameter(s)

In [11]:
Dataset.retrieve(name='high_low_exposure')

InvalidRequestError: no matching resource found with given parameter(s)